In [10]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"
import json
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, TimeDistributed, Dense, Activation, Embedding

#import all modules.

In [3]:
#attributes of batch
batchSize=16
seqLength=64

In [4]:
def makeBatch(allChars,uniqueChars):
    size=allChars.shape[0]
    batchCapacity=int(allChars/batchSize)
    
    for s in range(0, batchCapacity- seqLength, 64):
    
        X = np.zeros((batchSize, seqLength))
        Y = np.zeros((batchSize, seqLength, uniqueChars))
        
        for i in range(0,16):
            for j in range(0,64):
                X[i, j] = all_chars[i * batchCapacity + s + j]
                Y[i, j, allChars[i * batchCapacity + s + j + 1]] = 1 
        yield X, Y

In [5]:
def built_model(batch_size, seq_length, unique_chars):
    model = Sequential()
    
    model.add(Embedding(input_dim = unique_chars, output_dim = 512, batch_input_shape = (batch_size, seq_length), name = "embd_1")) 
    
    model.add(LSTM(256, return_sequences = True, stateful = True, name = "lstm_1"))
    model.add(Dropout(0.2, name = "drp_1"))
    
    model.add(LSTM(256, return_sequences = True, stateful = True,name = "lstm_2"))
    model.add(Dropout(0.2,name = "drp_2"))
    
    model.add(LSTM(256, return_sequences = True, stateful = True,name = "lstm_3"))
    model.add(Dropout(0.2,name = "drp_3"))
    
    model.add(TimeDistributed(Dense(unique_chars)))
    model.add(Activation("softmax"))
    
    # TODO : load weights.
    
    return model

In [6]:
def train_model(data, epochs = 40):
    #mapping character to index number
    cti = {ch: i for (i, ch) in enumerate(sorted(list(set(data))))}
    print("Number of unique characters included in our ABC database = {}".format(len(cti)))
    #Will print the number of different charachters in our database
    
    with open(os.path.join(data_dir, cij), mode = "w") as f:
        json.dump(cti, f)
        
    itc = {i: ch for (ch, i) in cti.items()}
    uc = len(cti)
    
    model = built_model(BATCH_SIZE, SEQ_LENGTH, uc)
    model.summary()
    model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    
    all_char = np.asarray([cti[c] for c in data], dtype = np.int32)
    print("Total number of characters = "+str(all_char.shape[0])) #155222
    
    epoch_number, loss, accuracy = [], [], []
    
    for epoch in range(epochs):
        print("Epoch {}/{}".format(epoch+1, epochs))
        final_epoch_loss, final_epoch_accuracy = 0, 0
        epoch_number.append(epoch+1)
        
        for i, (x, y) in enumerate(read_batches(all_chars, unique_chars)):
            final_epoch_loss, final_epoch_accuracy = model.train_on_batch(x, y) 
            #Training On Batch
            print("Batch: {}, Loss: {}, Accuracy: {}".format(i+1, final_epoch_loss, final_epoch_accuracy))
            #here, above we are reading the batches one-by-one and train our model on each batch one-by-one.
        loss.append(final_epoch_loss)
        accuracy.append(final_epoch_accuracy)
        
        #saving weights after every 10 epochs
        #Weights to be saved 4 times
        if (epoch + 1) % 10 == 0:
            if not os.path.exists(model_weights_directory):
                os.makedirs(model_weights_directory)
            model.save_weights(os.path.join(model_weights_directory, "Weights_{}.h5".format(epoch+1)))
            print('Saved Weights at epoch {} to file Weights_{}.h5'.format(epoch+1, epoch+1))
    
    #creating dataframe and record all the losses and accuracies at each epoch Total 40
    log_frame = pd.DataFrame(columns = ["Epoch", "Loss", "Accuracy"])
    log_frame["Epoch"] = epoch_number
    log_frame["Loss"] = loss
    log_frame["Accuracy"] = accuracy
    log_frame.to_csv("../Data/log.csv", index = False)
